## Visualization of knowledge graphs

*    This is a Google Colab notebook. You must have a Google account with a Google Drive to store/ load the main file with the relationships, for inspection. 
*    Upload the notebook from its location in GitHub and allow the code to access your Google Drive.
*    Please put your credentials in the cell "Connect to the Virtuoso database".
*    Put your own keywords in the cell with tile "Enter keywords below". Note that the search is case-sensitive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installations

In [2]:
!pip install pyodbc

     |████████████████████████████████| 280 kB 3.2 MB/s 
  Created wheel for pyodbc: filename=pyodbc-4.0.32-cp37-cp37m-linux_x86_64.whl size=287374 sha256=99f1d47d74c3c21ac244d6cac9cce00cfe1588a599e040e502757972d4cc8acf
  Stored in directory: /root/.cache/pip/wheels/2e/9c/da/8652fd42e0f662015554f00a9e96fe4f438dfd1ef59787879e
Successfully built pyodbc


In [3]:
!pip install pyvis

  Created wheel for pyvis: filename=pyvis-0.2.1-py3-none-any.whl size=23688 sha256=a6dff1272c4f6232365cf58fdc564421221d22c502e1878984a9b23c80a67991
  Stored in directory: /root/.cache/pip/wheels/2a/8f/04/6340d46afc74f59cc857a594ca1a2a14a1f4cbd4fd6c2e9306
Successfully built pyvis


In [4]:
!pip install SPARQLWrapper
!pip install sparql_dataframe

     |████████████████████████████████| 482 kB 4.9 MB/s 
     |████████████████████████████████| 41 kB 576 kB/s 


In [5]:
!apt-get install virtuoso-opensource

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libvirtodbc0 virtuoso-opensource-6.1 virtuoso-opensource-6.1-bin
  virtuoso-opensource-6.1-common virtuoso-server virtuoso-vad-conductor
  virtuoso-vsp-startpage
Suggested packages:
  virtuoso-vad-doc virtuoso-vad-demo virtuoso-vad-tutorial
  virtuoso-vad-rdfmappers virtuoso-vad-sparqldemo virtuoso-vad-syncml
  virtuoso-vad-bpel virtuoso-vad-isparql virtuoso-vad-ods virtuoso-vad-dbpedia
  virtuoso-vad-facetedbrowser
The following NEW packages will be installed:
  libvirtodbc0 virtuoso-opensource virtuoso-opensource-6.1
  virtuoso-opensource-6.1-bin virtuoso-opensource-6.1-common virtuoso-server
  virtuoso-vad-conductor virtuoso-vsp-startpage
0 upgraded, 8 newly installed, 0 to

## Imports

In [6]:
import os 
import re
import logging
import sys
import pyodbc
import hashlib
import pandas as pd
import numpy as np
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, POST, DIGEST, GET
from SPARQLWrapper import JSON, INSERT, DELETE
import sparql_dataframe

pd.set_option('display.max_rows', 500)

from IPython.display import Image

import random

import palettable
from palettable.colorbrewer.sequential import Blues_9
from palettable.colorbrewer.sequential import Greens_9
from palettable.colorbrewer.qualitative import Accent_8
from palettable.colorbrewer.qualitative import Paired_7

from pyvis import network as net
from IPython.core.display import display, HTML

## Connect to the Virtuoso database

In [7]:
def connect_db(DSN, DBA, UID, PWD):

    connection = pyodbc.connect('DRIVER=/usr/lib/odbc/virtodbc.so;HOST=lod.csd.auth.gr:1111;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW;DATABASE=ESTAT')
    cursor = connection.cursor()

    return connection, cursor


def connect_virtuoso(DSN, UID, PWD):

    sparql = SPARQLWrapper(DSN)
    sparql.setHTTPAuth(DIGEST)
    sparql.setCredentials(UID, PWD)
    sparql.setMethod(GET)

    return sparql


user = "kimon"
login = "RkhvQYZ442e2JVXLHdtW"
# Connection to CDB 
connection, cursor = connect_db('Virtuoso All', 
                                'ESTAT', 
                                user, 
                                login)

# Connection to the KDB 
endpoint = "http://lod.csd.auth.gr:8890/sparql/"
sparql = connect_virtuoso(endpoint, 
                          user, 
                          login)

## Extraction of relationships

In [8]:
def databaseTable(keywords):

  RelationsStatements = """
  DEFINE input:inference <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
  PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
  PREFIX estatdata: <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
  PREFIX oecd: <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/oecd/> 
  PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
  select ?article ?title ?articleURL ?r ?reftitle ?refURL where { 
      {?article rdf:type estat:Article .
      ?article ?p ?t .}
      UNION
      {?article rdf:type estat:Article .
      ?article estat:hasParagraph ?x.
      ?x ?p ?t .}
      filter contains(str(?t),\" """ + str(keywords) + """\") 
      ?article estat:title ?title .
      ?article estat:hasURL ?articleURL.
      ?article estat:hasReference ?r .
      {?r rdf:type estat:Table} 
      UNION
      {?r rdf:type estat:Database}
      ?r estat:hasURL ?refURL .
      ?r estat:title ?reftitle .
  } 
  """

  sparql.setQuery(RelationsStatements)
  sparql.method = "POST"
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()['results']['bindings']
  results = pd.json_normalize(results)
  
  return results

def articleTable(keywords):

  RelationsStatements = """
  DEFINE input:inference <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
  PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
  PREFIX estatdata: <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/>
  PREFIX oecd: <https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/oecd/> 
  PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
  select ?article ?title ?articleURL ?euroArticle ?euroTitle ?euroURL where { 
      {?article rdf:type estat:Article .
      ?article ?p ?t .}
      UNION
      {?article rdf:type estat:Article .
      ?article estat:hasParagraph ?x.
      ?x ?p ?t .}
      filter contains(str(?t),\" """ + str(keywords) + """\") 
      ?article estat:title ?title .
      ?article estat:hasURL ?articleURL.
      ?article estat:hasReference ?r .
      ?r estat:hasURI ?euroArticle.
      ?euroArticle estat:title ?euroTitle .
      ?euroArticle estat:hasURL ?euroURL.
  } 
  """
  
  sparql.setQuery(RelationsStatements)
  sparql.method = "POST"
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()['results']['bindings']
  results = pd.json_normalize(results)
  
  return results




### Production of the dataframe with the relationships

In [9]:

def produce_df(keywords):

    results1 = databaseTable(keywords)
    results2 = articleTable(keywords)

    listData = []
    for name, row in results1.iterrows():
        listData.append((row[1], row[3], row[6], row[8], row[10], row[13]))

    listArticle = []
    for name, row in results2.iterrows():
        listArticle.append((row[1], row[3], row[6], row[8], row[10], row[13]))

    finalList = listData + listArticle

    hashHelper = {}
    articleURI, articleURL,articletitle, relatedURI, relatedURL, relatedtitle = [], [], [], [], [], []
    for relation in finalList:
        articleURI.append(relation[0])
        articleURL.append(relation[2])
        articletitle.append(relation[1])
        relatedURI.append(relation[3])
        relatedURL.append(relation[5])
        relatedtitle.append(relation[4])
        hashHelper['articleURI'] = articleURI
        hashHelper['articleURL'] = articleURL
        hashHelper['articletitle'] = articletitle
        hashHelper['relatedURI'] = relatedURI
        hashHelper['relatedURL'] = relatedURL
        hashHelper['relatedTitle'] = relatedtitle

    # Create DataFrame  
    df = pd.DataFrame(hashHelper)
    #print(df)
    print(len(finalList))
    return df

### Enter keywords below
*    To re-run with different keywords change them below and re-run ("Runtime > Run after") from this cell. 

In [24]:
keywords = 'mortality statistics'

## Produce file for inspection

In [25]:

df = produce_df(keywords)
df.to_excel('/content/drive/MyDrive/df.xlsx')
display(df)

40


,articleURI,articleURL,articletitle,relatedURI,relatedURL,relatedTitle
0,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,http://www.oecd-ilibrary.org/social-issues-mig...,OECD Health at a Glance
1,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Excess mortality statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/web/population-d...,EUROPOP2019 Population projections at national...
2,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Weekly death statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,http://ec.europa.eu/eurostat/cache/metadata/en...,Weekly deaths
3,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Excess mortality statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/web/population-d...,EUROPOP2019 Population projections at national...
4,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Preventable and treatable mortality statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,http://ec.europa.eu/eurostat/web/health/overview,Health
5,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,http://eur-lex.europa.eu/legal-content/EN/TXT/...,Regulation EC No 1177 2003
6,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Weekly death statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,http://ec.europa.eu/eurostat/cache/metadata/en...,Weekly deaths
7,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,EU statistics on income and living conditions ...
8,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics
9,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Heating and cooling degree days statistics


In [26]:
## add column relatedType indicating the type of the related resource

conditions = [
    (df['relatedURL'].str.contains('Glossary:')),              
    (df['relatedURL'].str.contains('statistics-explained/index.php')),
    ## (df['relatedURL'].str.contains('data/database')),
    ## (df['relatedURL'].str.contains('/news/')),    
    (df['relatedURL'].str.contains('eurostat/product')),    
    (df['relatedURL'].str.contains('eur-lex.europa.eu'))    
    ]
##values = ['GL article','SE article', 'Data', 'News','Publication','Legislation']
values = ['GL article','SE article', 'Publication','Legislation']
df['relatedType'] = np.select(conditions,values,default='Other') 
df = df[df['relatedType'] != 'Other'].copy()
df.reset_index(drop=False,inplace=True)
df.drop(columns='index',inplace=True)
df.rename(columns={'articletitle':'articleTitle'},inplace=True)
df


,articleURI,articleURL,articleTitle,relatedURI,relatedURL,relatedTitle,relatedType
0,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,http://eur-lex.europa.eu/legal-content/EN/TXT/...,Regulation EC No 1177 2003,Legislation
1,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,EU statistics on income and living conditions ...,SE article
2,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,SE article
3,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Heating and cooling degree days statistics,SE article
4,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Services producer price index SPPI,GL article
5,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Main condition,GL article
6,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Levels of data,GL article
7,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Model,GL article
8,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Minimum wage,GL article
9,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Adult unemployment rate,GL article


In [27]:
## unique URIs in Articles and Related resources

valuesURI = df[['articleURI','relatedURI']].values
uniqueURI = np.unique(valuesURI)
print('uniqueURIs:')
uniqueURI

df2 = pd.DataFrame(uniqueURI,columns=['uniqueURI'])
df2['ID'] = range(len(df2))
df2

uniqueURIs:


,uniqueURI,ID
0,https://ec.europa.eu/eurostat/NLP4StatRef/know...,0
1,https://ec.europa.eu/eurostat/NLP4StatRef/know...,1
2,https://ec.europa.eu/eurostat/NLP4StatRef/know...,2
3,https://ec.europa.eu/eurostat/NLP4StatRef/know...,3
4,https://ec.europa.eu/eurostat/NLP4StatRef/know...,4
5,https://ec.europa.eu/eurostat/NLP4StatRef/know...,5
6,https://ec.europa.eu/eurostat/NLP4StatRef/know...,6
7,https://ec.europa.eu/eurostat/NLP4StatRef/know...,7
8,https://ec.europa.eu/eurostat/NLP4StatRef/know...,8
9,https://ec.europa.eu/eurostat/NLP4StatRef/know...,9


In [28]:
## transfer the unique IDs to the main file

df3 = pd.merge(df,df2,how='inner', left_on='articleURI',right_on='uniqueURI')
df3.rename(columns={'ID':'articleID'},inplace=True)
df3 = pd.merge(df3,df2,how='inner', left_on='relatedURI',right_on='uniqueURI')
df3.rename(columns={'ID':'relatedID'},inplace=True)
df3 = df3[['articleID','relatedID','relatedType','articleURI','articleURL','articleTitle','relatedURI','relatedURL','relatedTitle']]
df3

,articleID,relatedID,relatedType,articleURI,articleURL,articleTitle,relatedURI,relatedURL,relatedTitle
0,1,18,Legislation,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,http://eur-lex.europa.eu/legal-content/EN/TXT/...,Regulation EC No 1177 2003
1,1,0,SE article,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,EU statistics on income and living conditions ...
2,1,1,SE article,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics
3,6,1,SE article,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Preventable and treatable mortality statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics
4,1,9,SE article,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Heating and cooling degree days statistics
5,1,11,GL article,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Services producer price index SPPI
6,1,12,GL article,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Main condition
7,1,13,GL article,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Healthy life years statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Levels of data
8,2,13,GL article,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Excess mortality statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Levels of data
9,2,13,GL article,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Excess mortality statistics,https://ec.europa.eu/eurostat/NLP4StatRef/know...,https://ec.europa.eu/eurostat/statistics-expla...,Levels of data


In [29]:
## related type groups and assignment to colors
type_groups = np.unique(df3['relatedType'])
print(type_groups)
node_colors = Paired_7.hex_colors[:len(type_groups)]
node_colors

['GL article' 'Legislation' 'SE article']


['#A6CEE3', '#1F78B4', '#B2DF8A']

In [30]:
## Create graph

g=net.Network(height='600px', width='75%',heading='')
g.barnes_hut()
g.repulsion(node_distance=100, spring_length=200)

## First the unique articles - source nodes, by articleID 
uniqueArticles = df3.groupby(['articleID'])[['articleTitle']].agg(list).reset_index() ## aggregate into lists - titles are repeated
col_index = int(np.where(type_groups=='SE article')[0]) ## index of color to use for SE articles
for i in range(len(uniqueArticles)): ## for some reason, single elements must be put in a list, add_node() does not work
    g.add_nodes([uniqueArticles.loc[i,'articleID']], color=[node_colors[col_index]], label = [uniqueArticles.loc[i,'articleTitle'][0]])

## Then the related nodes, by type
relatedGroups = df3.groupby(['relatedType'])[['relatedID','relatedTitle']].agg(list).reset_index() ## again aggregate into lists
for i in range(len(relatedGroups)):
    how_many_nodes = len(relatedGroups.loc[i,'relatedID'])
    g.add_nodes(relatedGroups.loc[i,'relatedID'],label=relatedGroups.loc[i,'relatedTitle'], color=[node_colors[i]]*how_many_nodes)

## The edges
for i in range(len(df3)):        
    n1 = int(df3.loc[i,'articleID'])
    n2 = int(df3.loc[i,'relatedID'])
    type_ind = int(np.where(type_groups==df3.loc[i,'relatedType'])[0])
    col =  node_colors[type_ind]
    lbl = type_groups[type_ind]
    g.add_edge(n1,n2,color=col,label=lbl)
g.show('example.html')
display(HTML('example.html'))
